In [1]:
#Importing the standard libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [2]:
#Importing the solver modules
import system
from obe import obe
from states import SigmaLevel,PiLevelParity
from obe import Excitation
import time
import warnings
warnings.filterwarnings('ignore')

##### Defining the system. We construct a 2 level system where <br> the ground state is |G = 0.5, N = 2, F1 = 1.5, F = 2, mF = -1> <br> the excited state is |J = 0.5-, F1 = 0.5, F = 1, mF = -1>. <br> Even though the example here treats the case of 138BaF, the states are written in the Hund's case $b_{\beta S}$ basis

In [3]:
b=system.System([],[]) 

#We will generate a ground state first. In a BaF molecule it is a Sigma level
b.sigma_states.append(SigmaLevel(0.5,2,3/2,2,-1))     #state0 = (G,N,F1,F,mF)
 
#The excited state is a Pi level
b.pi_states.append(PiLevelParity(-1,1/2,1/2,1,-1))    #state2 = (parity,J,F1,F,mF)


In [4]:
print(b.sigma_states)
print(b.pi_states)

[|G = 0.5, N = 2, F1 = 1.5, F = 2, mF = -1>]
[|J = 0.5-, F1 = 0.5, F = 1, mF = -1>]


In [5]:
#Next we will generate the Hamiltonian for this two level system
b.sigma_Hamiltonian.generate_bare()
b.pi_Hamiltonian.generate_bare()
b.sigma_Hamiltonian.diagonalize()
b.pi_Hamiltonian.diagonalize()

In [6]:
G = b.sigma_Hamiltonian.diagonalized_states
GH = b.sigma_Hamiltonian.diagonalized_Hamiltonian
E =b.pi_Hamiltonian.diagonalized_states
EH = b.pi_Hamiltonian.diagonalized_Hamiltonian
b.generate_interaction_Hamiltonian(G,E)
b.generate_branching_ratios(G,E)

Pi branching took : 0.3010411262512207 sec
Sigma- branching took : 0.3133969306945801 sec
Sigma+ branching took : 0.3033621311187744 sec


In [7]:
b.branching_ratios

array([[1.]])

In [8]:
#Constructing the Hamiltonian
H0 = np.zeros((len(G)+len(E),len(G)+len(E)))
H0[0:len(G),0:len(G)] = GH
H0[len(G):,len(G):] = EH

Hint=b.interaction_Hamiltonian

n0 = [1/len(G)]*len(G)+[0]*len(E)
Hint

array([[ 0.        +0.j, -0.28867513+0.j],
       [-0.28867513-0.j,  0.        +0.j]])

In [9]:
Gamma = 2*np.pi*2.7

#### Rabi oscillation when atom interacts with a pulse of light. <br> The pulse is a rectangular function smootened at the transtions extending between $-2t_{\sigma}$ and $+2t_{\sigma}$

In [14]:
#Define the em wave interacting with the atom
#By default we are using a z-polarized light
rabi_prime = 50.0 #this is the overall factor that will multiply the matrix element to produce the actual rabi freqiemncy for the transtion
tsigma = 10/Gamma/4 #4t_sigma is time that corresponds to the 1/e^2 diameter of the beam


#Define the light field
field = Excitation(rabi_prime,pol = 0,ground_state=G[0],excited_state=E[0],detuning = 0,position=0, diameter=tsigma*4,shape = "Uniform") 
rabi = rabi_prime*Hint[0,1]
print(f"Actual rabi frequency : {rabi}") 

#Constructing the optical bloch equation system
my_obe = obe(field,[G,E],H0,Hint,b.branching_ratios)

#initializing the density matrix
steps=5000
r_init = np.array([[1.0,0],[0,0]]).astype(np.complex128)

#calling the solve function of the obe class
tim,ans = my_obe.solve(steps,r_init,max_step_size = 1/Gamma, package= 'Python')

#Populatiton of the excited state as a function of time
pop = ans[:,3]
plt.plot(tim,pop);
plt.xlabel('Time (us)')
plt.ylabel('Population in the excited state');


Actual rabi frequency : (-14.433756729740649+0j)


2025-01-28 14:02:03.208 python[12183:523166] error messaging the mach port for IMKCFRunLoopWakeUpReliable


#### Rabi oscillation by varying pulse area

In [16]:
#Observing Rabi fluorescence by varying pulse width
steps=5000
rabi_prime=100
tsigma_array = np.linspace(1e-6/Gamma,2.0/Gamma,1000)

pulse_area = np.abs(np.sqrt(4*np.pi)*tsigma_array*2*np.pi*rabi_prime*Hint[0,1])
r22  = []

for tsigma in tsigma_array:
    field = Excitation(rabi_prime,pol = 0,ground_state=G[0],excited_state=E[0],detuning = 0,position=0, diameter=tsigma*4,shape = "Gaussian")
    my_obe = obe([field],[G,E],H0,Hint,b.branching_ratios)
    
    r_init = np.zeros(np.shape(H0)).astype(np.complex128)
    for i in range(len(G)):
        r_init[i,i] = 1.0/len(G)
    
    t,ans = my_obe.solve(steps,r_init,max_step_size = 1/Gamma)
    r22.append(np.sum(ans[:,3])*(t[1]-t[0])/(t[-1]-t[0]))
    #print(tsigma*Gamma)

In [ ]:
plt.plot(pulse_area/np.pi,r22,'o-')
plt.xlabel("Pulse area (in units of π)")
plt.ylabel("Integrated fluorsecence (arb unit)");

#### Frequency scan around resonance

In [13]:
#Scan

def r22(rabi_prime, det):
    
    field = Excitation(rabi_prime,0,G[0],E[0],detuning = det,position=0, diameter=40/Gamma,shape = "Gaussian")
    
    my_obe = obe([field],[G,E],H0,Hint,b.branching_ratios)
    
    steps=5000
    #t=np.linspace(0,30.0/Gamma,steps)
    r_init = np.zeros(np.shape(H0)).astype(np.complex128)
    for i in range(len(G)):
        r_init[i,i] = 1.0/len(G)
    t,ans = my_obe.solve(steps,r_init,max_step_size = 0.1/Gamma)
    return np.sum(ans[:,3])*(t[1]-t[0])/(t[-1]-t[0])


rabi_prime = 2
detuning = np.linspace(-20,20,100)
int_fl = []
for det in detuning:
    int_fl.append(r22(rabi_prime,det))

In [ ]:
plt.plot(detuning,int_fl)
plt.xlabel("Detuning from resonace (MHz)")
plt.ylabel("Integrated fluorescence ")